#  step1 : prepare data

In [3]:
from sklearn.preprocessing import StandardScaler
from bigdl.chronos.data.repo_dataset import get_public_dataset

def get_tsdata():
    name = 'nyc_taxi'
    tsdata_train, tsdata_valid, tsdata_test = get_public_dataset(name)
    stand_scaler = StandardScaler()
    for tsdata in [tsdata_train, tsdata_valid, tsdata_test]:
        tsdata.impute(mode="linear")\
              .scale(stand_scaler, fit=(tsdata is tsdata_train))
    return tsdata_train, tsdata_valid, tsdata_test

tsdata_train, tsdata_valid, tsdata_test = get_tsdata()

train_data = tsdata_train.roll(lookback=20, horizon=1).to_numpy()
val_data = tsdata_valid.roll(lookback=20, horizon=1).to_numpy()
test_data = tsdata_test.roll(lookback=20, horizon=1).to_numpy()

# step2: define TCNForecaster

In [6]:
from bigdl.chronos.forecaster.tcn_forecaster import TCNForecaster

import bigdl.nano.automl.hpo.space as space
forecaster = TCNForecaster(past_seq_len=20,
                        future_seq_len=1,
                        input_feature_num=1,
                        output_feature_num=1,
                        kernel_size=space.Categorical(4, 5),
                        num_channels=space.Categorical([16, 16], [30]*7),
                        loss="mse",
                        seed=2021,
                        metrics=['mae', 'mse', 'mape'],
                        lr=0.01)
forecaster.num_processes = 1

Global seed set to 2021


# step3 : normal HyperParameter Optimization

In [10]:
forecaster.tune(data=train_data, validation_data=val_data,
                target_metric='mse',
                direction='minimize',
                n_trials=4,
                epochs=3)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[I 2022-07-26 09:35:29,674] A new study created in memory with name: no-name-3013d5bf-76e6-419f-baf9-f1f647c370e9


Starting a new tuning


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Training: 258it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[I 2022-07-26 09:36:55,726] Trial 3 finished with value: 0.9993957281112671 and parameters: {'kernel_size▁choice': 1, 'num_channels▁choice': 1}. Best is trial 0 with value: 0.04218398779630661.


In [12]:
forecaster.fit(train_data, epochs=3)

/opt/anaconda3/envs/nano/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/nano/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:59: LightningDeprecationWarning: Setting `Trainer(flush_logs_every_n_steps=10)` is deprecated in v1.5 and will be removed in v1.7. Please configure flushing in the logger instead.
  f"Setting `Trainer(flush_logs_every_n_steps={flush_logs_every_n_steps})` is deprecated in v1.5 "

  | Name  | Type            | Params
----------

/opt/anaconda3/envs/nano/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py
<torch.utils.data.dataloader.DataLoader object at 0x7effe1314250> None


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [13]:
forecaster.evaluate(val_data)

[array([[0.128252]], dtype=float32),
 array([[0.03106654]], dtype=float32),
 array([[0.80104655]], dtype=float32)]

# step4 : Muti-Objective HPO with built-in latency metric

In [31]:
forecaster = TCNForecaster(past_seq_len=20,
                        future_seq_len=1,
                        input_feature_num=1,
                        output_feature_num=1,
                        kernel_size=space.Categorical(4, 5),
                        num_channels=space.Categorical([16, 16], [30]*7),
                        loss="mse",
                        seed=2021,
                        metrics=['mae', 'mse', 'mape'],
                        lr=0.01)
forecaster.num_processes = 1

trials = forecaster.tune(data=train_data, validation_data=val_data,
                target_metric=['mse', 'latency'],
                direction=None,
                directions=['minimize', 'minimize'],
                n_trials=4,
                epochs=3)

Global seed set to 2021
/opt/anaconda3/envs/nano/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[I 2022-07-26 10:05:14,657] A new study created in memory with name: no-name-1ebc1535-fb3b-4cfd-b430-8be8bd8440bf


Starting a new tuning


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Training: 258it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[I 2022-07-26 10:06:18,897] Trial 3 finished with values: [0.03345833718776703, 3.7870226364869337] and parameters: {'kernel_size▁choice': 0, 'num_channels▁choice': 0}. 


In [38]:
import optuna
optuna.visualization.plot_pareto_front(forecaster.tune_trainer.hposearcher.study, target_names=["mse", "latency"])

/opt/anaconda3/envs/nano/lib/python3.7/site-packages/ipykernel_launcher.py:2: ExperimentalWarning:

plot_pareto_front is experimental (supported from v2.4.0). The interface can change in the future.



In [35]:
trials

[FrozenTrial(number=2, values=[0.032849498093128204, 3.8183462894879856], datetime_start=datetime.datetime(2022, 7, 26, 10, 5, 56, 703666), datetime_complete=datetime.datetime(2022, 7, 26, 10, 6, 7, 865302), params={'kernel_size▁choice': 1, 'num_channels▁choice': 0}, distributions={'kernel_size▁choice': CategoricalDistribution(choices=(0, 1)), 'num_channels▁choice': CategoricalDistribution(choices=(0, 1))}, user_attrs={}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None),
 FrozenTrial(number=3, values=[0.03345833718776703, 3.7870226364869337], datetime_start=datetime.datetime(2022, 7, 26, 10, 6, 7, 866057), datetime_complete=datetime.datetime(2022, 7, 26, 10, 6, 18, 897597), params={'kernel_size▁choice': 0, 'num_channels▁choice': 0}, distributions={'kernel_size▁choice': CategoricalDistribution(choices=(0, 1)), 'num_channels▁choice': CategoricalDistribution(choices=(0, 1))}, user_attrs={}, system_attrs={'nsga2:generation': 0

# step5 : Muti-Objective HPO with built-in accelarated latency metric

In [40]:
forecaster = TCNForecaster(past_seq_len=20,
                        future_seq_len=1,
                        input_feature_num=1,
                        output_feature_num=1,
                        kernel_size=space.Categorical(4, 5),
                        num_channels=space.Categorical([16, 16], [30]*7),
                        loss="mse",
                        seed=2021,
                        metrics=['mae', 'mse', 'mape'],
                        lr=0.01)
forecaster.num_processes = 1

trials = forecaster.tune(data=train_data, validation_data=val_data,
                target_metric=['mse', 'latency'],
                direction=None,
                directions=['minimize', 'minimize'],
                n_trials=4,
                epochs=3,
                acceleration=True)

Global seed set to 2021
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[I 2022-07-26 10:17:31,606] A new study created in memory with name: no-name-730d7217-bc46-45ed-95e6-a8793b5895b2


Starting a new tuning


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         Validate metric                     DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             latency                      7.197476350344145
             val/loss                     0.9392467737197876
     val/mean_absolute_error              0.7819218039512634
val/mean_absolute_percentage_error        2.678788423538208
      val/mean_squared_error              0.9392467737197876
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
#########
val _DataLoaderSource(instance=<torch.utils.data.dataloader.DataLoader object at 0x7effd7be5c50>, name='val_dataloader')


Validation: 0it [00:00, ?it/s]

[I 2022-07-26 10:19:10,645] Trial 3 finished with values: [0.9392468333244324, 6.854513113315289] and parameters: {'kernel_size▁choice': 1, 'num_channels▁choice': 1}. 


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         Validate metric                     DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             latency                      6.854513113315289
             val/loss                     0.9392468333244324
     val/mean_absolute_error              0.7819218635559082
val/mean_absolute_percentage_error        2.678788185119629
      val/mean_squared_error              0.9392468333244324
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## visualization

In [44]:
import optuna
optuna.visualization.plot_pareto_front(forecaster.tune_trainer.hposearcher.study, target_names=["mse", "accelarated latency"])

## find corresponding acceleration by user_attrs

In [54]:
trials[1].user_attrs

{'optimization': 'openvino'}